In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models,datasets, transforms
from mevit_model import MultiExitViT
from tqdm import tqdm


transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])


test_dataset = datasets.ImageNet(root='./data', train=True, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(len(test_dataset))

Files already downloaded and verified
50000
